In [1]:
from simulation_modules import chatbot_sim as cs
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
import json
from pprint import pprint

In [14]:
# load the dataset {'gender': gender, 'name': name}
cli_manager = cs.ClientManager("simulation_modules/client_names.jsonl") # loads the entire dataset
cli_data = cli_manager.get_sample(absolute=40) # retrieve 20 random names
train, test = train_test_split(cli_data, test_size=0.2, random_state=42)
print("Train size:", len(train), "Test size:", len(test), ". Expected total conversations (all possible fluxes): ", len(train)*12)

Train size: 32 Test size: 8 . Expected total conversations (all possible fluxes):  384


In [15]:
pprint(test)

[{'gender': 'm', 'name': 'Leonardo Silva'},
 {'gender': 'm', 'name': 'Óscar René Guzmán'},
 {'gender': 'm', 'name': 'Iván Darío Castillo'},
 {'gender': 'm', 'name': 'Daniel Alexander García'},
 {'gender': 'f', 'name': 'Camila Martínez'},
 {'gender': 'm', 'name': 'Diego Emiliano Ramírez'},
 {'gender': 'f', 'name': 'Natalia Herrera'},
 {'gender': 'm', 'name': 'Mauricio Andrade'}]


In [16]:
# random dates start
s_date = datetime(2024, 1, 1)

# random dates end
e_date = datetime.now()

agent_names = ["Raúl"]
bank_name = "Banco Azteca"

dates_generator = cs.DatesGenerator()

train_entries_generator = cs.RandomEntryGenerator("random_day", agent_names, train, bank_name, dates_generator)
test_entries_generator = cs.RandomEntryGenerator("current_day", agent_names, test, bank_name, dates_generator)

train_entries = train_entries_generator.generate_random_entries(s_date, e_date)
test_entries = test_entries_generator.generate_random_entries(s_date, e_date)

pprint(test_entries)

[{'amount_pesos': 29261,
  'ask_for_payment_day': 'miércoles',
  'bank_name': 'Banco Azteca',
  'client_name': {'gender': 'm', 'name': 'Leonardo Silva'},
  'current_date': '10 de diciembre de 2024',
  'days_late': 53,
  'name_of_the_agent': 'Raúl',
  'system_current_date_time': 'martes 2024-12-10 10:29 AM',
  'system_current_datetime_object': datetime.datetime(2024, 12, 10, 10, 29, 35, 945924),
  'system_tomorrow_date': 'miércoles 2024-12-11',
  'tomorrow_date': '11 de diciembre de 2024'},
 {'amount_pesos': 44853,
  'ask_for_payment_day': 'miércoles',
  'bank_name': 'Banco Azteca',
  'client_name': {'gender': 'm', 'name': 'Óscar René Guzmán'},
  'current_date': '10 de diciembre de 2024',
  'days_late': 49,
  'name_of_the_agent': 'Raúl',
  'system_current_date_time': 'martes 2024-12-10 10:29 AM',
  'system_current_datetime_object': datetime.datetime(2024, 12, 10, 10, 29, 35, 945924),
  'system_tomorrow_date': 'miércoles 2024-12-11',
  'tomorrow_date': '11 de diciembre de 2024'},
 {'amou

In [17]:
# create an instance of the AztecaBot
bot = cs.AztecaBot()
print(bot.default_chat_flux)

[[True, True, True, False, False, False, False, False, False, False, False], [True, True, False, False, False, False, False, False, False, False, False], [True, True, False, True, False, False, False, False, False, False, False], [True, False, False, False, True, True, False, False, False, False, False], [True, False, False, False, True, False, False, False, False, False, False], [True, False, False, False, True, False, True, False, False, False, False], [True, False, False, False, False, False, False, True, True, False, False], [True, False, False, False, False, False, False, True, False, False, False], [True, False, False, False, False, False, False, True, False, True, False], [True, False, False, False, False, False, False, False, False, False, False], [False, False, False, False, False, False, False, False, False, False, True], [False, False, False, False, False, False, False, False, False, False, False]]


Create the train dataset for gpt-4o-mini fine tuning

In [18]:
cs.FineTuningDataset.create_dataset(
    entries=train_entries,
    chatbot=bot,
    chat_flux=bot.default_chat_flux,
    generate_weights=True, #all weights have the value of 1 by default
    filename="train_azteca_v4.jsonl"
)

Dataset created successfully and saved in train_azteca_v4.jsonl


Create the test dataset for gpt-4o-mini fine tuning

In [9]:
cs.FineTuningDataset.create_dataset(
    entries=test_entries,
    chatbot=bot,
    chat_flux=bot.default_chat_flux,
    filename="test_azteca_v4.jsonl"
)

Dataset created successfully and saved in test_azteca_v4.jsonl


### Analyze Assistant Responses Patterns

Perform an analysis of the amount of responses given an 'assistant' pattern. If there are more responses of certain assistant pattern there is more probability that the model learns quickly that specific assistant response.

In [10]:
patterns = bot.assistant_patterns
pattern_statistics = cs.FineTuningDataset.analyze_dataset_patterns("train_azteca_v1.jsonl", patterns)

In [11]:
print("Pattern statistics:", pattern_statistics)

Pattern statistics: {'greeting': 384, 'primary_info': 320, 'amount_reconfirmation': 288, 'final_reconfirmation': 192, 'ask_for_tomorrow_pay': 192, 'contact_you_later': 128, 'first_attempt_agreement': 224, 'second_attempt_agreement': 128, 'ask_for_line_holder': 64, 'ask_for_callback': 32, 'wrong_person': 32}


### Configure the weights of the Assistant Responses
Configure the weights for OpenAI chat completion format: 0 for a content that is no longer needed in order to train the model; 1 for a content that the model still needs to learn. In example the 'greeting' pattern appears in 100% of the conversations, its spected that the model learns 

In [19]:
assistant_thresholds_config = {
    "greeting": 1,  # After 70% of the dataset, set weight to 0. Set the 30% of the dataset with weight = 0
    "primary_info": 1,  # After 70% of the dataset, set weight to 0. Set the 30% of the dataset with weight = 0
    "amount_reconfirmation": 1, # After 100% of the dataset, set weight to 0. Set the 0% of the dataset with weight = 0
    "final_reconfirmation": 1, 
    "ask_for_tomorrow_pay": 1,
    "contact_you_later": 1,
    "first_attempt_agreement": 1,
    "second_attempt_agreement": 1,
    "ask_for_line_holder": 1,
    "ask_for_callback": 1,
    "wrong_person": 1
}
patterns = bot.assistant_patterns

In [15]:
cs.FineTuningDataset.adjust_weights_by_line(
    input_file="train_azteca_v1.jsonl", 
    output_file="train_azteca_weight.jsonl", 
    thresholds=assistant_thresholds_config, 
    patterns=patterns
)

Dataset processed successfully. Saved to train_azteca_weight.jsonl.
